In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import glob
import PyPDF2 
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from PyPDF2 import PdfReader

/home/cptaswadu/RESCUE-n8n/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [3]:
class RAGPolicyRetriever:
    def __init__(self, policy_folder_path, openai_api_key):
        self.policy_folder_path = policy_folder_path
        self.policies = {}
        self.embeddings = {}
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.llm_client = OpenAI(api_key=openai_api_key)

    def load_policies(self):
        pdf_files = glob.glob(os.path.join(self.policy_folder_path, "*.pdf"))
        for pdf_file in pdf_files:
            with open(pdf_file, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() or ""
            self.policies[os.path.basename(pdf_file)] = text
        print(f"✅ Loaded {len(self.policies)} policies.")

    def embed_policies(self):
        for doc_name, doc_text in self.policies.items():
            self.embeddings[doc_name] = self.embedder.encode([doc_text])[0]
        print("✅ Embeddings created.")

    def extract_insurance_and_test(self, patient_info):
        insurance_provider = None
        if "United Healthcare" in patient_info or "UnitedHealthcare" in patient_info:
            insurance_provider = "UnitedHealthcare"
        elif "Aetna" in patient_info:
            insurance_provider = "Aetna"
        elif "Cigna" in patient_info:
            insurance_provider = "Cigna"
        elif "Blue Cross" in patient_info:
            insurance_provider = "Blue Cross"
        elif "CapitalBC" in patient_info:
            insurance_provider = "CapitalBC"
        elif "BCBS" in patient_info:
            insurance_provider = "BCBS"
        elif "Medicaid" in patient_info:
            insurance_provider = "Medicaid"

        if "Whole exome sequencing" in patient_info or "WES" in patient_info:
            test_name = "Whole Exome Sequencing"
        elif "Whole genome sequencing" in patient_info or "WGS" in patient_info:
            test_name = "Whole Genome Sequencing"
        elif "BRCA1" in patient_info or "BRCA2" in patient_info:
            test_name = "BRCA1/BRCA2"
        elif "Panel" in patient_info:
            test_name = "Panel Testing"
        else:
            test_name = "Genetic Testing"

        return insurance_provider, test_name

    def filter_policies_by_insurance(self, insurance_name):
        filtered = {}
        for doc_name, doc_text in self.policies.items():
            if insurance_name and insurance_name.replace(" ", "").lower() in doc_name.replace(" ", "").lower():
                filtered[doc_name] = doc_text
        return filtered

    def find_top_policies(self, patient_info, insurance_name, top_k=5):
        filtered_policies = self.filter_policies_by_insurance(insurance_name)

        if not filtered_policies:
            print("❗ No policies matched the insurance. Using all policies.")
            filtered_policies = self.policies

        query_embedding = self.embedder.encode([patient_info])[0]
        scored_policies = []
        for doc_name, doc_text in filtered_policies.items():
            doc_embedding = self.embeddings[doc_name]
            score = cosine_similarity([query_embedding], [doc_embedding])[0][0]
            scored_policies.append((doc_name, score, doc_text))

        scored_policies.sort(key=lambda x: x[1], reverse=True)
        return scored_policies[:top_k]

    def rerank_policies(self, patient_info, candidates):
        candidate_texts = [c[2][:500].replace("\n", " ") for c in candidates]

        prompt = f"""You are an expert insurance policy assistant.

You will be given patient information and a list of candidate insurance policies.
Please select which candidate policy best matches the patient's situation.

Patient Information:
{patient_info}

Candidate Policies:"""

        for idx, text in enumerate(candidate_texts, 1):
            prompt += f"\n\nPolicy {idx}:\n{text}"

        prompt += """

Please answer with only the number of the most appropriate policy.
Do not explain. Just output the number.

Answer:"""

        response = self.llm_client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": prompt}],
            temperature=0
        )

        result = response.choices[0].message.content.strip()
        match = re.search(r'(\d+)', result)
        selected_idx = int(match.group(1)) - 1 if match else 0

        return candidates[selected_idx]

    def find_policy(self, patient_info):
        insurance, test = self.extract_insurance_and_test(patient_info)
        print(f"📌 Detected insurance: {insurance}, test: {test}")

        candidates = self.find_top_policies(patient_info, insurance)

        print("📋 Top candidates:")
        for doc_name, score, _ in candidates:
            print(f"- {doc_name}: {score:.4f}")

        best_policy = self.rerank_policies(patient_info, candidates)
        return best_policy[0], best_policy[2]

In [4]:
class QnAExecutor:
    def __init__(self, questions_list):
        self.questions_list = questions_list
        self.formatted_questions = self.format_questions()

    def format_questions(self):
        formatted_questions = []
        for q in self.questions_list:
            question_line = f"{q['id']}. {q['question']}"
            if q.get("options") == ["Free text answer"]:
                question_line += "\n(Free text answer allowed.)"
            else:
                question_line += f"\nOptions: {', '.join(q['options'])}"
                if "additional_if_yes" in q:
                    question_line += f"\nIf you answer 'Yes', ALSO select from: {', '.join(q['additional_if_yes'])}"
                if "additional_if_no" in q:
                    question_line += f"\nIf you answer 'No', ALSO select from: {', '.join(q['additional_if_no'])}"
            formatted_questions.append(question_line)

        return "\n\n".join(formatted_questions)

    def clean_json_response(self, content):
        content = re.sub(r"```(?:json)?", "", content).strip()
        if content.endswith("```"):
            content = content[:-3].strip()
        return content

    def run_qna(self, patient_info, policy_name, policy_text, case_id):
        prompt = f"""
You are a clinical insurance assistant specializing in genetic testing coverage policies.

You will be given:

1. Patient clinical information (including their insurance provider, plan type, and state of residence).
2. Official insurance policy document text (strictly use this policy content for insurance coverage decision making).

Instructions:

- Answer all questions strictly based on the insurance policy document provided.
- Do NOT refer to general guidelines or policies from other insurance providers.
- If policy document does not clearly specify rules, you MAY use patient's clinical information to infer answers carefully.
- Do NOT assume coverage criteria from other insurers or general clinical guidelines unless explicitly stated in the policy.
- Output answers in JSON format ONLY.

==== PATIENT INFORMATION ====
{patient_info}

==== INSURANCE POLICY DOCUMENT (from URL: {policy_name}) ====
{policy_text}

==== QUESTIONS ====
{self.formatted_questions}

Output your answers in JSON format only and include the policy_url at the end.
"""

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a clinical insurance assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        result_content = response.choices[0].message.content.strip()
        result_json = {}

        try:
            cleaned_content = self.clean_json_response(result_content)
            result_json = json.loads(cleaned_content)

            final_result = {}
            for k, v in result_json.items():
                if k == "policy_url":
                    continue

                if "_selection" in k or "_details" in k:
                    base_key = k.replace("_selection", "").replace("_details", "")
                    final_result[f"{base_key}_followup"] = [v] if isinstance(v, str) else v
                else:
                    final_result[k] = v

            save_dir = "/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG"
            os.makedirs(save_dir, exist_ok=True)
            filename = os.path.join(save_dir, f"{case_id}_qna_result.json")
            with open(filename, "w") as f:
                json.dump(final_result, f, indent=2)

            print(f"✅ QnA result saved to {filename}")

        except Exception as e:
            print("❗ JSON parsing error:", e)
            final_result = {
                "error": "JSON parsing failed",
                "raw_content": result_content
            }

        print("QnA Result JSON:", final_result)
        return final_result

In [5]:
questions_file_path = "/home/cptaswadu/RESCUE-n8n/insurance/dataset/Insurance_Genetic_Testing_QA.json"

with open(questions_file_path, "r") as f:
    questions_data = json.load(f)

qna_executor = QnAExecutor(questions_data["questions"])

In [6]:
case_ex = [
    {
        "id": "Case1",
        "patient_info": "An 8-year-old boy with neurodevelopmental delay and seizures. A prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by the genetic counselor to investigate potential underlying genetic causes that may guide diagnosis and future treatment decisions. There is also a family history of neurodevelopmental disorders, as his older brother was diagnosed with autism spectrum disorder. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case2",
        "patient_info": "An 8-year-old boy with mild learning difficulties and no significant neurological symptoms. There is no family history of genetic conditions, and his prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by his primary care provider (PCP) to explore potential genetic factors as part of general health screening and educational planning. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    },
    {
        "id": "Case3",
        "patient_info": "A 35-year-old woman with a strong family history of breast and ovarian cancer. Her mother was diagnosed with breast cancer at age 42, and her maternal aunt had ovarian cancer in her 50s. The patient herself has no history of cancer but has dense breast tissue and is considered at increased risk. The genetic counselor has recommended BRCA1/BRCA2 testing to assess her hereditary cancer risk and guide risk-reducing management decisions, including potential prophylactic options. The patient is covered by Aetna Open Access Managed Choice Plan and resides in California."
    },
    {
        "id": "Case4",
        "patient_info": "A 28-year-old woman with no family history of breast or ovarian cancer. The patient requested BRCA1/BRCA2 genetic testing after reading about genetic risks online. There were no prior specialist consultations or referrals, and no other clinical risk factors have been identified. The test was ordered directly by her primary care physician at the patient's request. The patient is enrolled in Aetna Open Access Managed Choice Plan and lives in Texas."
    }
]

In [7]:
retriever = RAGPolicyRetriever(policy_folder_path="/home/cptaswadu/RESCUE-n8n/insurance/insurance_policy", openai_api_key = openai_api_key)
retriever.load_policies()
retriever.embed_policies()

✅ Loaded 585 policies.
✅ Embeddings created.


In [8]:
results = {}

for case in case_ex:
    case_id = case["id"]
    patient_info = case["patient_info"]

    print(f"\n==== Running QnA for {case_id} ====")

    # 정책 선택
    policy_name, policy_text = retriever.find_policy(patient_info)
    print(f"Selected Policy: {policy_name}")

    # QnA 실행
    result = qna_executor.run_qna(patient_info, policy_name, policy_text, case_id)

    results[case_id] = result
    print(result)


==== Running QnA for Case1 ====
📌 Detected insurance: UnitedHealthcare, test: Whole Exome Sequencing


📋 Top candidates:
- United Healthcare_whole-exome-and-whole-genome-sequencing.pdf: 0.4831
- United Healthcare_genetic-testing-neuromuscular-disorders.pdf: 0.4247
- United Healthcare_genetic-testing-hereditary-cancer.pdf: 0.4086
- United Healthcare_preimplantation-genetic-testing.pdf: 0.3903
- United Healthcare_genetic-testing-cardiac-disease.pdf: 0.3890
Selected Policy: United Healthcare_whole-exome-and-whole-genome-sequencing.pdf
✅ QnA result saved to /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/Case1_qna_result.json
QnA Result JSON: {'Q0': 'Whole Exome Sequencing (WES)', 'Q1': 'Yes', 'Q2': 'Yes', 'Q3': 'Yes', 'Q4': 'Yes', 'Q4_followup': ['ACMG'], 'Q5': 'Yes', 'Q6': 'No', 'Q7': 'Yes', 'Q8': 'No', 'Q9': 'Yes', 'Q9_followup': ['Yes'], 'Q10': 'No', 'Q10_followup': ['Diagnostic'], 'Q11': 'Yes', 'Q12': 'Yes', 'Q13': 'Not specified', 'Q14': 'Yes', 'Q14_followup': ['81415'], 'Q15': 'No', 'Q16': 'Yes', 'Q17': 'To submit the claim, ensure that the test is ordered by an approved spe

In [9]:
print(results)

{'Case1': {'Q0': 'Whole Exome Sequencing (WES)', 'Q1': 'Yes', 'Q2': 'Yes', 'Q3': 'Yes', 'Q4': 'Yes', 'Q4_followup': ['ACMG'], 'Q5': 'Yes', 'Q6': 'No', 'Q7': 'Yes', 'Q8': 'No', 'Q9': 'Yes', 'Q9_followup': ['Yes'], 'Q10': 'No', 'Q10_followup': ['Diagnostic'], 'Q11': 'Yes', 'Q12': 'Yes', 'Q13': 'Not specified', 'Q14': 'Yes', 'Q14_followup': ['81415'], 'Q15': 'No', 'Q16': 'Yes', 'Q17': 'To submit the claim, ensure that the test is ordered by an approved specialist, confirm that the patient meets the medical necessity criteria, and provide documentation of prior genetic counseling. Include the appropriate CPT code (81415) and any relevant medical records that support the necessity of the test.'}, 'Case2': {'Q0': 'Whole Exome Sequencing (WES)', 'Q1': 'Yes', 'Q2': 'No', 'Q3': 'No', 'Q4': 'No', 'Q5': 'Yes', 'Q6': 'No', 'Q7': 'No', 'Q8': 'Yes', 'Q8_description': 'A single gene or targeted gene panel should be performed prior to determining if WES is necessary.', 'Q9': 'No', 'Q10': 'Yes', 'Q10_d

In [10]:
ground_truth = {
    "Case1": {
        "Q0": "Whole Exome Sequencing (WES)",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "Yes",
        "Q4": "Yes",
        "Q4_followup": [
      "ACMG"
    ],
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "Yes",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "Yes"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Diagnostic"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81415", "81416"
    ],
        "Q15": "No",
        "Q16": "Yes"
  },
  
    "Case2": {
        "Q0": "Whole Exome Sequencing (WES)",
        "Q1": "Yes",
        "Q2": "No",
        "Q3": "No",
        "Q4": "No",
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "No"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Other"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81415", "81416"
    ],
        "Q15": "No",
        "Q16": "No"
  },
  
    "Case3": {
        "Q0": "BRCA1/BRCA2 genetic testing",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "Yes",
        "Q4": "Yes",
        "Q4_followup": [
      "NCCN"
    ],
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "Yes"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Risk Assessment"
    ],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81162"
    ],
        "Q15": "No",
        "Q16": "Yes"
  },
  
    "Case4": {
        "Q0": "BRCA1/BRCA2 genetic testing",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "No",
        "Q4": "No",
        "Q5": "No",
        "Q6": "No",
        "Q7": "No",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": [
      "No"
    ],
        "Q10": "No",
        "Q10_followup": [
      "Risk Assessment"
    ],
        "Q11": "No",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": [
      "81162"
    ],
        "Q15": "No",
        "Q16": "No"
  }
    
}

In [11]:
def evaluate_qna_result(case_id, predicted_result, gold_result):
    records = []
    correct_count = 0
    total_count = 0

    for qid in gold_result:
        if not qid.startswith("Q") or qid == "policy_url" or qid == "Q17" or "_followup" in qid:
            continue

        pred_answer = predicted_result.get(qid, "").strip()
        gold_answer = gold_result.get(qid, "").strip()

        is_correct = pred_answer == gold_answer
        score = 1 if is_correct else 0

        records.append({
            "Case": case_id,
            "Question": qid,
            "Predicted": pred_answer,
            "Gold": gold_answer,
            "Score": score
        })

        total_count += 1
        correct_count += score

        # follow-up 평가
        followup_key = qid + "_followup"
        pred_followup = predicted_result.get(followup_key, None)
        gold_followup = gold_result.get(followup_key, None)

        if is_correct and gold_followup is not None:

            def normalize(ans):
                if ans is None:
                    return "None"
                if isinstance(ans, list):
                    return ", ".join(ans)
                return ans

            pred_followup_norm = normalize(pred_followup)
            gold_followup_norm = normalize(gold_followup)

            followup_score = 1 if pred_followup_norm == gold_followup_norm else 0

            records.append({
                "Case": case_id,
                "Question": followup_key,
                "Predicted": pred_followup_norm,
                "Gold": gold_followup_norm,
                "Score": followup_score
            })

            total_count += 1
            correct_count += followup_score

    accuracy = correct_count / total_count * 100 if total_count > 0 else 0

    records.append({
        "Case": case_id,
        "Question": "TOTAL",
        "Predicted": f"Correct: {correct_count}",
        "Gold": f"Incorrect: {total_count - correct_count}",
        "Score": f"Accuracy: {accuracy:.2f}%"
    })

    df_records = pd.DataFrame(records)
    df_records
    return df_records

evaluate_qna_result('Case1', results['Case1'], ground_truth['Case1'])

,Case,Question,Predicted,Gold,Score
0,Case1,Q0,Whole Exome Sequencing (WES),Whole Exome Sequencing (WES),1
1,Case1,Q1,Yes,Yes,1
2,Case1,Q2,Yes,Yes,1
3,Case1,Q3,Yes,Yes,1
4,Case1,Q4,Yes,Yes,1
5,Case1,Q4_followup,ACMG,ACMG,1
6,Case1,Q5,Yes,Yes,1
7,Case1,Q6,No,No,1
8,Case1,Q7,Yes,Yes,1
9,Case1,Q8,No,No,1


In [12]:
output_csv_path = "/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/RAG_qna_eval_results.csv"

output_dir = os.path.dirname(output_csv_path)
os.makedirs(output_dir, exist_ok=True)

def evaluate_all_cases(policy_records, gold_answers, output_csv_path):
    all_dfs = []
    
    policy_records = results
    for case_id in policy_records:
        pred_result = policy_records[case_id]
        gold_result = gold_answers.get(case_id)

        if pred_result is None or gold_result is None:
            print(f"Skipping {case_id} due to missing data.")
            continue

        df_case = evaluate_qna_result(case_id, pred_result, gold_result)
        all_dfs.append(df_case)

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        final_df.to_csv(output_csv_path, index=False)
        print(f"✅ Evaluation completed and saved to {output_csv_path}")
        print(final_df)
        return final_df
    else:
        print("❗ No records to save.")
        return pd.DataFrame()

evaluate_all_cases(results, ground_truth, output_csv_path)


✅ Evaluation completed and saved to /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/RAG/RAG_qna_eval_results.csv
     Case      Question                     Predicted  \
0   Case1            Q0  Whole Exome Sequencing (WES)   
1   Case1            Q1                           Yes   
2   Case1            Q2                           Yes   
3   Case1            Q3                           Yes   
4   Case1            Q4                           Yes   
..    ...           ...                           ...   
78  Case4           Q14                           Yes   
79  Case4  Q14_followup                          None   
80  Case4           Q15                            No   
81  Case4           Q16                            No   
82  Case4         TOTAL                   Correct: 13   

                            Gold             Score  
0   Whole Exome Sequencing (WES)                 1  
1                            Yes                 1  
2                            Yes      

,Case,Question,Predicted,Gold,Score
0,Case1,Q0,Whole Exome Sequencing (WES),Whole Exome Sequencing (WES),1
1,Case1,Q1,Yes,Yes,1
2,Case1,Q2,Yes,Yes,1
3,Case1,Q3,Yes,Yes,1
4,Case1,Q4,Yes,Yes,1
...,...,...,...,...,...
78,Case4,Q14,Yes,Yes,1
79,Case4,Q14_followup,None,81162,0
80,Case4,Q15,No,No,1
81,Case4,Q16,No,No,1
